In [1]:

#dane
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#nauka
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import itertools
import graphviz
import lightgbm as lgb
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline


#Wczytaj dane
colnames=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothenss_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dismension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dismension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compatness_worst','concavity_worst','concave poinst_worst','summetry_worst','fractal_dismension_worst']
df = pd.read_csv('Desktop/dane_do_mgr.csv',names=colnames, header=None)



In [2]:

#WYSWIETL 10 WIERSZY DANYCH
df.head(10)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothenss_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compatness_worst,concavity_worst,concave poinst_worst,summetry_worst,fractal_dismension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440
6,844359,M,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368
7,84458202,M,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510
8,844981,M,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720
9,84501001,M,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750


In [3]:

#WYSWIETL I POLICZ ILE JEST ZLOSLIWYCH, ILE ŁAGODNYCH ODMIAN RAKA W KOLUMNIE DIAGOZA
df['diagnosis'].value_counts()


B    357
M    212
Name: diagnosis, dtype: int64

In [4]:

#SPRAWDŹ CZY SĄ JAKIEŚ ZERA W ZBIORACH ( NIEPOŻĄDANE WARTOSCI )
#JESLI SĄ TO ZSUMUJ ILE TAKICH MIEJSC MAMY
df.isnull().sum()
df.isna().sum()


id                          0
diagnosis                   0
radius_mean                 0
texture_mean                0
perimeter_mean              0
area_mean                   0
smoothenss_mean             0
compactness_mean            0
concavity_mean              0
concave points_mean         0
symmetry_mean               0
fractal_dismension_mean     0
radius_se                   0
texture_se                  0
perimeter_se                0
area_se                     0
smoothness_se               0
compactness_se              0
concavity_se                0
concave points_se           0
symmetry_se                 0
fractal_dismension_se       0
radius_worst                0
texture_worst               0
perimeter_worst             0
area_worst                  0
smoothness_worst            0
compatness_worst            0
concavity_worst             0
concave poinst_worst        0
summetry_worst              0
fractal_dismension_worst    0
dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder_Y = LabelEncoder()
test = labelencoder_Y.fit_transform(df["diagnosis"])


In [6]:
X = df.drop("diagnosis",axis=1)
Y = test

In [7]:
'''
1). X_train - Obejmuje to wszystkie zmienne niezależne, zostaną one użyte do trenowania modelu, 
również ponieważ określiliśmy test_size = 0.4, oznacza to, że 60% obserwacji z pełnych 
danych zostanie użytych do trenowania/dopasowania modelu oraz 40 % z reszty zostanie użyty do przetestowania modelu.

2). X_test — Jest to pozostała część 40% niezależnych zmiennych z danych, które nie 
zostaną wykorzystane w fazie uczenia i zostaną wykorzystane do predykcji w celu przetestowania dokładności modelu.

3). y_train — jest to twoja zmienna zależna, która musi być przewidziana przez ten 
model, obejmuje to etykiety kategorii względem twoich zmiennych niezależnych, musimy 
określić naszą zmienną zależną podczas uczenia/dopasowywania modelu.

4). y_test — te dane mają etykiety kategorii dla danych testowych. Etykiety te 
będą używane do testowania dokładności między kategoriami rzeczywistymi i przewidywanymi.
'''


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [8]:
# DLUGOSCI:
    # WSZYSTKIE DANE
    # DANE X_train
    # DANE X_test
    # DANE Y_train
    # DANE Y_test
len(X), len(X_train), len(X_test), len(Y_train), len(Y_test)

(569, 426, 143, 426, 143)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)


In [10]:
Y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
Y_test


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [12]:
X_test


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothenss_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compatness_worst,concavity_worst,concave poinst_worst,summetry_worst,fractal_dismension_worst
512,915691,13.40,20.52,88.64,556.7,0.11060,0.14690,0.14450,0.08172,0.2116,...,16.41,29.66,113.30,844.4,0.15740,0.38560,0.51060,0.20510,0.3585,0.11090
457,9112367,13.21,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,...,14.35,34.23,91.29,632.9,0.12890,0.10630,0.13900,0.06005,0.2444,0.06788
439,909410,14.02,15.66,89.59,606.5,0.07966,0.05581,0.02087,0.02652,0.1589,...,14.91,19.31,96.53,688.9,0.10340,0.10170,0.06260,0.08216,0.2136,0.06710
298,892214,14.26,18.17,91.22,633.1,0.06576,0.05220,0.02475,0.01374,0.1635,...,16.22,25.26,105.80,819.7,0.09445,0.21670,0.15650,0.07530,0.2636,0.07676
37,854941,13.03,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.02923,0.1467,...,13.30,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,88299702,23.21,26.97,153.50,1670.0,0.09509,0.16820,0.19500,0.12370,0.1909,...,31.01,34.51,206.00,2944.0,0.14810,0.41260,0.58200,0.25930,0.3103,0.08677
113,864292,10.51,20.19,68.64,334.2,0.11220,0.13030,0.06476,0.03068,0.1922,...,11.16,22.75,72.62,374.4,0.13000,0.20490,0.12950,0.06136,0.2383,0.09026
527,91813702,12.34,12.27,78.94,468.5,0.09003,0.06307,0.02958,0.02647,0.1689,...,13.61,19.27,87.22,564.9,0.12920,0.20740,0.17910,0.10700,0.3110,0.07592
76,8610629,13.53,10.94,87.91,559.2,0.12910,0.10470,0.06877,0.06556,0.2403,...,14.08,12.49,91.36,605.5,0.14510,0.13790,0.08539,0.07407,0.2710,0.07191


In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
cm

array([[86,  4],
       [31, 22]], dtype=int64)

In [14]:
# outcome values order in sklearn
#tp, fn, fp, tn = confusion_matrix(Y_test,Y_pred,labels=[1,0]).reshape(-1)
#print('Outcome values : \n', tp, fn, fp, tn)


from sklearn.metrics import classification_report
# classification report for precision, recall f1-score and accuracy
matrix = classification_report(Y_test,Y_pred,labels=[1,0])
print('Classification report : \n',matrix)


Classification report : 
               precision    recall  f1-score   support

           1       0.85      0.42      0.56        53
           0       0.74      0.96      0.83        90

    accuracy                           0.76       143
   macro avg       0.79      0.69      0.69       143
weighted avg       0.78      0.76      0.73       143

